# Exercise 8: Sound transformations

In this exercise you will use the HPS model to creatively transform sounds. There are two parts in this exercise. In the first one you should perform a natural sounding transformation on the speech sound that you used in the previous exercise (E7). In the second part you should select a sound of your choice and do a "creative" transformation. You will have to write a short description of the sound and of the transformation you did, giving the link to the original sound and uploading several transformed sounds.

For this exercise, you can use the `transformations_GUI.py` (in `software/transformations_interface/`) to try things, once decided you can fill up the code in this file. You can also do everything from here and add any new code you wish.

In order to perform a good/interesting transformation you should make sure that you have performed an analysis that is adequate for the type of transformation you want to do. Not every HPS analysis representation will work for every type of sound transformation. There will be things in the analysis that when modified will result in undesired artifacts. In general, for any transformation, it is best to have the harmonic values as smooth and continuous as possible and an stochastic representation as smooth and with as few values as possible. It might be much better to start with an analysis representation that does not result in the best reconstruction in exchange of having smoother and more compact data.

To help you with the exercise, we give a brief description of the transformation parameters used by the HPS transformation function:

1. `freqScaling`: frequency scaling factors to be applied to the harmonics of the sound, in time-value pairs (where value of 1 is no scaling). The time values can be normalized, from 0 to 1, or can correspond to the times in seconds of the input sound. The scaling factor is a multiplicative factor, thus a value of 1 is no change. Example: to transpose an octave the sound you can specify `[0, 2, 1, 2]`.
2. `freqStretching`: frequency stretching factors to be applied to the harmonics of the sound, in time-value pairs (value of 1 is no stretching). The time values can be normalized, from 0 to 1, or can correspond to the times in seconds of the input sound. The stretching factor is a multiplicative factor whose effect depend on the harmonic number, higher harmonics being more affected that lower ones, thus resulting in an inharmonic effect. A value of 1 results in no transformation. Example: an array like `[0, 1.2, 1, 1.2]` will result in a perceptually large inharmonic effect.
3. `timbrePreservation`: 1 preserves the original timbre, 0 does not. It can only have a value of 0 or of 1. By setting the value to 1 the spectral shape of the original sound is preserved even when the frequencies of the sound are modified. In the case of speech it would correspond to the idea of preserving the identity of the speaker after the transformation.
4. `timeScaling`: time scaling factors to be applied to the whole sound, in time-value pairs (value of 1 is no scaling). The time values can be normalized, from 0 to 1, or can correspond to the times in seconds of the input sound. The time scaling factor is a multiplicative factor, thus 1 is no change. Example: to stretch the original sound to twice the original duration, we can specify `[0, 0, 1, 2]`.

All the transformation values can have as many points as desired, but they have to be in the form of an array with time-value pairs, so of even size. For example a good array for a frequency stretching of a sound that has a duration of 3.146 seconds could be: `[0, 1.2, 2.01, 1.2, 2.679, 0.7, 3.146, 0.7]`.

## Part 1. Perform natural sounding transformations of a speech sound

Use the HPS model with the sound `speech-female.wav`, available in the sounds directory, to first analyze and then obtain a natural sounding transformation of the sound. The synthesized sound should sound as different as possible to the original sound while sounding natural. By natural we mean that it should sound like speech, that it could have been possible to be produced by a human, and by listening we should consider it as a speech sound, even though we might not be able to understand it. You should first make sure that you start from a good analysis, then you can do time and/or frequency scaling transformations. The transformation should be done with a single pass, no mixing of sounds coming from different transformations. Since you used the same sound in A7, use that experience to get a good analysis, but consider that the analysis, given that we now want to use it for applying a very strong transformation, might be done differently than what you did in A7.

Write a short paragraph for every transformation, explaining what you wanted to obtain and explaining the transformations you did, giving both the analysis and transformation parameter values (sufficiently detailed for the evaluator to be able to reproduce the analysis and transformation).

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import get_window
import sys, os
sys.path.append('../software/models/')
sys.path.append('../software/transformations/')
import utilFunctions as UF
import stft as STFT
import hpsModel as HPS
import hpsTransformations as HPST
import harmonicTransformations as HT
import IPython.display as ipd

In [2]:
# 1.1 perform an analysis/synthesis using the HPS model

input_file = '../sounds/speech-female.wav'

### set the parameters
window ='hamming'
M = 1261
N = 2048
t = -90
minSineDur = 0.05
nH = 65 
minf0 = 100
maxf0 = 250
f0et = 4
harmDevSlope = 0.01
stocf = 0.2

# no need to modify anything after this
Ns = 512
H = 128

(fs, x) = UF.wavread(input_file)
w = get_window(window, M, fftbins=True)
hfreq, hmag, hphase, stocEnv = HPS.hpsModelAnal(x, fs, w, N, H, t, nH, minf0, maxf0, f0et, harmDevSlope, minSineDur, Ns, stocf)
y, yh, yst = HPS.hpsModelSynth(hfreq, hmag, hphase, stocEnv, Ns, H, fs)

ipd.display(ipd.Audio(data=x, rate=fs))
ipd.display(ipd.Audio(data=y, rate=fs))


### Explain your parameter choices
In order to satisfy the requirements and to obtain a good analysis of the sound, we first set a STFT analysis and look for the pitch range and maximum number of harmonics by setting the required parameters and visualizing the spectrogram. A Hamming window  of size 1261 is chosen after trying different parameter combinations, and the FFT size will be the next power of 2 greater than the window size, (N =2048). We choose a hop size of 128 as it gives a good output result. To obtain the window size value we use the given formula, and we choose a close number to the result that it returns.

After listening to to the residual component of the HPR model and listening to the HPS model we will be able to choose the value of the reamining features: 
- *Window type, window size, FFT size:* Since their result with the STFT was good we decide to keep the same parameters for the HpS model.
- *Minimum f0 and Maximum f0:* After taking a close look to the STFT spectrogram we decide that a good but compact range for the f0 estimation would be the one selected minf0=100Hz and maxf0=250Hz.
- *Error threshold in f0 detection:* This parameter was slowly reduced to from 10 to the actual value to assure we keep the relevant f0 components.
- *Number of harmonics:* As we should only capture the relevant harmonics, after using the given formula, we've tested this value in order to only select the ones that affect the perceptual quality of the reconstruction.
- *Stochastic decimation factor:* We started with a decimation factor of 0.2 and modified it in order to get a compact and smooth function. At the end, the selected value was 0.2 as it gave a pretty good output.

In [3]:
# 1.2 Perform a transformation from the previous analysis

### define the transformations
freqScaling = np.array([0, 0.4, 2.01, 3.2, 3.146, 3.5])
freqStretching = np.array([0, 1, 2.01, 1, 2.679, 1.5, 3.146, 2.3])
timbrePreservation = 1
timeScaling = np.array([0, 0, 2.138, 2.138, 3.146, 2.8])


# no need to modify the following code 
Ns = 512
H = 128

# frequency scaling of the harmonics 
hfreqt, hmagt = HT.harmonicFreqScaling(hfreq, hmag, freqScaling, freqStretching, timbrePreservation, fs)

# time scaling the sound
yhfreq, yhmag, ystocEnv = HPST.hpsTimeScale(hfreqt, hmagt, stocEnv, timeScaling)

# synthesis from the trasformed hps representation 
y, yh, yst = HPS.hpsModelSynth(yhfreq, yhmag, np.array([]), ystocEnv, Ns, H, fs)

ipd.display(ipd.Audio(data=y, rate=fs))


### Explain your transformations
The election of the different parameters was made in order to try different transformations. 
In the very first part of the sound I tried to perform a frequency scaling to make a much more low sound, with no time scaling. After this, I tried to do the opposite thing, this is, modifying the frequency to a much higher one, while slowly increasing the time scaling towards the end.
The timbre is preserved and the parameters are selected in this way to assure a natural sounding of the output transformation.

### Explain Part 1
In this part, an analysis of the speech of a female and the following transformations were made in order to achieve a different sound output with the most quality posible. The value of the chosen parameters is explained in the different texts.

## Part 2. Perform creative transformations with a sound of your choice

Pick any natural and harmonic sound from Freesound and use the HPS model to do the most creative and interesting transformation you can come up with. Sounding as different as possible from the original sound.

It is essential that you start with a natural harmonic sound. Examples include (but not limited to) any acoustic harmonic instrument, speech, harmonic sound from nature, etc. As long as they have a harmonic structure, you can use it. You can even reuse the sound you used in A7-Part2 or upload your own sound to freesound and then use it.

The sound from Freesound to use could be in any format, but to use the sms-tools software you will have to first convert it to be a monophonic file (one channel), sampling rate of 44100, and 16bits samples.

You can do any interesting transformation with a single pass. It is not allowed to mix sounds obtained from different transformations. The transformed sound need not sound natural. So, time to show some creativity!

Write a short paragraph for every transformation, explaining what you wanted to obtain and explaining the transformations you did, giving both the analysis and transformation parameter values (sufficiently detailed for the evaluator to be able to reproduce the analysis and transformation).

In [5]:
# 1.1 perform an analysis/synthesis using the HPS model

### set the parameters
input_file = '../sounds/clean-guitar-lick.wav' 
window ='blackman'
M = 1323
N = 2048
t = -80
minSineDur = 0.05
nH = 147 
minf0 = 150
maxf0 = 570
f0et = 4
harmDevSlope = 0.01
stocf = 0.1

# no need to modify anything after this
Ns = 512
H = 128

(fs, x) = UF.wavread(input_file)
w = get_window(window, M, fftbins=True)
hfreq, hmag, hphase, stocEnv = HPS.hpsModelAnal(x, fs, w, N, H, t, nH, minf0, maxf0, f0et, harmDevSlope, minSineDur, Ns, stocf)
y, yh, yst = HPS.hpsModelSynth(hfreq, hmag, hphase, stocEnv, Ns, H, fs)

ipd.display(ipd.Audio(data=x, rate=fs))
ipd.display(ipd.Audio(data=y, rate=fs))

### Explain your parameter choices
Different parameter choicws were made in order to obtain a good analysis of the sound.
As in the previous case, a STFT analysis was made in order to set the required parameters (pitch range and maximum number of harmonics) thanks to the spectrogram visualization. A Blackman window was chosen in this case, and after using the given formula to calculate the window size, the chosen value for this parameter will be 1323, setting the FFT size in the next greater power of two, that is 2048. 




After listening to to the residual component of the HPR model and listening to the HPS model we will be able to choose the value of the reamining features: 
- *Window type, window size, FFT size:* Since their result with the STFT was good we decide to keep the same parameters for the HpS model.
- *Minimum f0 and Maximum f0:* After taking a close look to the STFT spectrogram we decide that a good but compact range for the f0 estimation would be the one selected minf0 = 150 Hz and maxf0 = 570 Hz.
- *Error threshold in f0 detection:* This parameter was slowly reduced to from 10 to the actual value to assure we keep the relevant f0 components.
- *Number of harmonics:* As we should only capture the relevant harmonics, after using the given formula, we've tested this value in order to only select the ones that affect the perceptual quality of the reconstruction.
- *Stochastic decimation factor:* We started with a decimation factor of 0.2 and modified it in order to get a compact and smooth function. At the end, the selected value was 0.1 as it gave a pretty good output.

In [6]:
# 1.2 Perform a transformation from the previous analysis

### define the transformations
freqScaling = np.array([0, 1.2, 2.01, 1.2, 2.679, .7, 3.146, .7])
freqStretching = np.array([0, 1, 2.679, 1.5, 3.146, 1.5])
timbrePreservation = 1
timeScaling = np.array([0, 0, 0.1, 1.5, 2.138, 2.138, 3.68, 0.4])


# no need to modify anything after this
Ns = 512
H = 128

# frequency scaling of the harmonics 
hfreqt, hmagt = HT.harmonicFreqScaling(hfreq, hmag, freqScaling, freqStretching, timbrePreservation, fs)

# time scaling the sound
yhfreq, yhmag, ystocEnv = HPST.hpsTimeScale(hfreqt, hmagt, stocEnv, timeScaling)

# synthesis from the trasformed hps representation 
y, yh, yst = HPS.hpsModelSynth(yhfreq, yhmag, np.array([]), ystocEnv, Ns, H, fs)

ipd.display(ipd.Audio(data=y, rate=fs))

### Explain your transformations
In order to achieve a very different sound compared to the existing one, both frequency scaling and stretching and also time scaling were performed.
Although the timbre was preserved, the different transformation assure that the resulting melody differs from the existing one, as at the beginning the time was scaled very quickly, in order to reduce its value at the end of the sound. In fact, it's pretty visible that the duration of the sound is half of the existing one, 2s compared to the 4s.
The frequency was modified in order to assure a different mood, even mixing some frequencies to achieve a very "chaotic" sound.